In [1]:
from src import gravix2
import tempfile

In [2]:
import matplotlib.pyplot as plt
import numpy as np

import nbfigtulz as ftl

In [3]:
img_dir = tempfile.TemporaryDirectory()
ftl.config['img_dir'] = img_dir.name
ftl.config['img_dir']

'/tmp/tmpwk_4ls_5'

In [4]:
libgravix2 = gravix2.load_library('libs/libgravix2.so')
planets = libgravix2.new_planets([(10., 20.), (-20., -30)])
missiles = libgravix2.new_missiles(1)

In [5]:
def shoot(missile, *, n=2):
    x = np.empty((0, 3))
    v = np.empty((0, 3))
    
    premature = False
    while n > 0 and not premature:
        n -= 1
        premature = missile.propagate(planets, h=1e-3)
        if premature:
            print('!!! PREMATURE !!!')
        
        x = np.concatenate((x, missile.trajectory.x), axis=0)
        v = np.concatenate((v, missile.trajectory.v), axis=0)
    
    return x, v

In [6]:
m = missiles[0]
m.launch(planets=planets, planet_idx=0, v=200., psi=90.)

x, v = shoot(m)
x.shape, v.shape

((200, 3), (200, 3))

In [7]:
m = missiles[0]
m.set(pos=x[-1], vel=-v[-1])

x2, v2 = shoot(m)
x2.shape, v2.shape

((200, 3), (200, 3))

In [8]:
@ftl.with_context
def make_fig(x, planets):
    lat = np.rad2deg(np.arcsin(x[:, 2]))
    lon = np.rad2deg(np.arctan2(x[:, 0], x[:, 1]))

    fig, ax = plt.subplots()
    ax.plot(lon, lat)

    ax.grid()
    ax.set_xlabel('Longitude (deg.)')
    ax.set_ylabel('Latitude (deg.)')

    for plat, plon in planets:
        ax.plot(plon, plat, 'o', c='black', alpha=.5)
        
    return ftl.save_fig(fig, 'trajectories')
        
        
make_fig(x, planets.planet_pos)

trajectories.png

In [9]:
@ftl.with_context
def make_fig(x1, x2):
    lat1 = np.rad2deg(np.arcsin(x1[:, 2]))
    lon1 = np.rad2deg(np.arctan2(x1[:, 0], x1[:, 1]))
    
    lat2 = np.rad2deg(np.arcsin(x2[:, 2]))
    lon2 = np.rad2deg(np.arctan2(x2[:, 0], x2[:, 1]))

    fig, ax = plt.subplots()
    ax.plot(lat1 - lat2[::-1], label=r'$\Delta \phi$')
    ax.plot(lon1 - lon2[::-1], label=r'$\Delta \lambda$')
    
    ax.legend()
    ax.set_xlabel('Time step')
    ax.set_ylabel('Error')
    
    return ftl.save_fig(fig, 'diff')
    
    
make_fig(x, x2)

diff.png